# Demonstration of how to execute a swap transaction with retry logic:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.blockchain import retry_swap_tokens, convert_to_base_unit, get_wallet_contents
from src.wallet_secrets import get_wallet_keypair, get_wallet_address, WalletType
from src.token_addresses import SOL, USDC
from dotenv import load_dotenv
import os

### I will swap 0.001 SOL for USDC in one of my bot base wallets:

In [ ]:
BOT_IDX = 0 # Bot #0
# First, get the address and keypair for the base wallet:
load_dotenv()
base_mnemonic = os.getenv("SOLAMBO_BASE")
base_keypair = get_wallet_keypair(WalletType.BOT, base_mnemonic, wallet_idx=BOT_IDX) 
base_address = get_wallet_address(WalletType.BOT, base_mnemonic, wallet_idx=BOT_IDX)
print(base_keypair, base_address)

5Jw7rqPE97NhEasJeUw95WWhJcMEc7w8bda2E14EmKYxUR9TcY4vzn9rQZtz1uv1BddpKpd6vpmSMLKPiBEHDs7i 3cSvQygypScaJb2bJHfdzXwwWmayvNin37Go8AkSTysS


In [4]:
# Next define the input and output mints and trade parameters:
# This will Swap 0.001 SOL with a 1% slippage and 50000 LAMPORT fee:
output_mint=USDC
input_mint=SOL
sol_amount = 0.001 # amount of SOL
slippage_bps = 100
prioritization_fee = 50000 # this is important for getting a trade in a trade! 
base_amount = convert_to_base_unit(input_mint, sol_amount)
print(f"Amount of lamports to swap {base_amount}") # 

Amount of lamports to swap 1000000


**Execute the SWAP** 

This first example will execute a single trade and wait for confirmation:

In [5]:
# This will try to swap up to 3 times, with a delay of 5s between each try
transaction_dict = await retry_swap_tokens(input_mint, 
                output_mint, 
                base_amount, 
                base_keypair, 
                slippage_bps, 
                prioritization_fee, 
                max_retries=3, 
                retry_delay=5)


In [ ]:
# You should wait for a bit for the trade to settle before moving on (at least 10s)

In [6]:
transaction_dict

{'signature': '3g15fRN8pJ2b2g2u1Ain9ESLCbEQfJWYEQzZ1srnCnmk78cJ3kW3VZytUyVKkZCZC7FaKfGKsmn82Muv1WrAKxPE',
 'quote': {'inputMint': 'So11111111111111111111111111111111111111112',
  'inAmount': '1000000',
  'outputMint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
  'outAmount': '177155',
  'otherAmountThreshold': '175384',
  'swapMode': 'ExactIn',
  'slippageBps': 100,
  'platformFee': None,
  'priceImpactPct': '0',
  'routePlan': [{'swapInfo': {'ammKey': 'GMgh4NtWrGaUf1RR2kcXD7LY1jou1qFAuSsQeKp5ow4a',
     'label': 'Meteora DLMM',
     'inputMint': 'So11111111111111111111111111111111111111112',
     'outputMint': 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v',
     'inAmount': '1000000',
     'outAmount': '177155',
     'feeAmount': '161',
     'feeMint': 'So11111111111111111111111111111111111111112'},
    'percent': 100}],
  'contextSlot': 340021227,
  'timeTaken': 0.001338427,
  'swapUsdValue': '0.1770468058109381959875917808',
  'simplerRouteUsed': False,
  'mostReliableAmmsQuote

In [7]:
# Now swap back:
wallet_contents = await get_wallet_contents(base_address)
amount_USDC = wallet_contents.get(USDC,{}).get('raw_amount',0)
print(f"Swapping {amount_USDC} USDC")
input_mint = USDC
output_mint = SOL
transaction_dict = await retry_swap_tokens(input_mint, 
                output_mint, 
                amount_USDC, 
                base_keypair, 
                slippage_bps, 
                prioritization_fee, 
                max_retries=3, 
                retry_delay=5)


Swapping 177071 USDC
